In [1]:
# Install required packages
!pip install crewai crewai-tools gradio openai arxiv scholarly beautifulsoup4 requests

import os
import gradio as gr
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool
from langchain_openai import ChatOpenAI
import arxiv
import scholarly
from typing import List, Dict
import json

  DEPRECATION: Building 'bibtexparser' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'bibtexparser'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'free-proxy' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'free-proxy'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'sgmllib3k' using the legacy setup.py bdist_

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached Deprecated-1.2.18-py2.py3-none-any.whl.metadata (5.7 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached selenium-4.34.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
Using cached Deprecated-1.2.18-py2.py3-none-any.whl (10.0 kB)
   --

C:\Users\soumi\AppData\Roaming\Python\Python312\site-packages\pydantic\fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [2]:
# Initialize the LLM
llm = ChatOpenAI(
    model="gpt-4",
    temperature=0.7
)

In [3]:
# Define custom tools for research
class ArxivSearchTool:
    def search_papers(self, query: str, max_results: int = 5) -> List[Dict]:
        """Search for papers on arXiv"""
        search = arxiv.Search(
            query=query,
            max_results=max_results,
            sort_by=arxiv.SortCriterion.Relevance
        )

        papers = []
        for result in search.results():
            papers.append({
                "title": result.title,
                "authors": [author.name for author in result.authors],
                "abstract": result.summary,
                "url": result.entry_id,
                "published": str(result.published)
            })
        return papers

In [4]:
# Initialize tools
arxiv_tool = ArxivSearchTool()

In [5]:
# Define Agents
topic_explainer = Agent(
    role='Topic Explainer',
    goal='Explain complex research topics in clear, accessible language and provide comprehensive overviews',
    backstory="""You are an expert educator with deep knowledge across multiple research domains.
    You excel at breaking down complex concepts into understandable explanations while maintaining accuracy.
    You understand the importance of providing context and connecting ideas to help researchers grasp new topics quickly.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

literature_finder = Agent(
    role='Literature Finder',
    goal='Find and summarize relevant research papers, identifying key contributions and methodologies',
    backstory="""You are a seasoned research librarian with expertise in academic literature search.
    You know how to identify the most relevant and impactful papers in any field. You're skilled at
    summarizing papers concisely while capturing their key contributions, methodologies, and findings.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

gap_analyzer = Agent(
    role='Gap Analyzer',
    goal='Identify research gaps, unexplored areas, and suggest promising research directions',
    backstory="""You are a research strategist with years of experience in identifying opportunities
    in academic research. You excel at spotting patterns, finding gaps in existing literature,
    and proposing innovative research directions that could advance the field.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

In [6]:
def create_tasks(research_topic: str):
    explain_topic_task = Task(
        description=f"""Provide a comprehensive explanation of the research topic: {research_topic}
        Include:
        1. Core concepts and definitions
        2. Historical context and evolution
        3. Current state of the field
        4. Key challenges and open questions
        5. Interdisciplinary connections""",
        agent=topic_explainer,
        expected_output="A detailed explanation of the research topic with all requested components"
    )

    find_literature_task = Task(
        description=f"""Search and analyze literature related to: {research_topic}
        For each paper found:
        1. Provide title, authors, and publication year
        2. Summarize the main contributions
        3. Explain the methodology used
        4. Highlight key findings
        5. Note limitations mentioned by authors
        Use the arxiv_tool to search for papers.""",
        agent=literature_finder,
        expected_output="A comprehensive literature review with summaries of relevant papers"
    )

    analyze_gaps_task = Task(
        description=f"""Based on the topic explanation and literature review for {research_topic}, identify:
        1. Research gaps in the current literature
        2. Unexplored or underexplored areas
        3. Methodological limitations in existing work
        4. Potential interdisciplinary opportunities
        5. Suggest 3-5 specific research questions that could address these gaps
        6. Provide a research outline for the most promising direction""",
        agent=gap_analyzer,
        expected_output="A detailed gap analysis with specific research suggestions and an outline"
    )

    return [explain_topic_task, find_literature_task, analyze_gaps_task]


In [7]:
def create_research_crew(research_topic: str):
    tasks = create_tasks(research_topic)

    crew = Crew(
        agents=[topic_explainer, literature_finder, gap_analyzer],
        tasks=tasks,
        process=Process.sequential,
        verbose=True
    )

    return crew


In [8]:
def run_research_companion(research_topic: str, specific_focus: str = None):
    """Run the research companion for a given topic"""

In [9]:
def run_research_companion(research_topic: str, specific_focus: str = None):
    """Run the research companion for a given topic"""
    # Modify topic if specific focus is provided
    if specific_focus and specific_focus.strip():
        full_topic = f"{research_topic} with focus on {specific_focus}"
    else:
        full_topic = research_topic

In [10]:
def gradio_interface(research_topic, specific_focus, api_key):
    """Gradio interface function"""

    # Update API key if provided
    if api_key and api_key.strip():
        os.environ["OPENAI_API_KEY"] = api_key

    if not research_topic:
        return "Please enter a research topic.", "", "", ""

    # Run the research companion
    results = run_research_companion(research_topic, specific_focus)
    if "error" in results:
        error_msg = f"Error: {results['error']}"
        return error_msg, error_msg, error_msg, error_msg

    # Format outputs
    explanation = results.get("explanation", results.get("full_output", ""))
    literature = results.get("literature_review", "")
    gaps = results.get("gap_analysis", "")

    # Create formatted output
    formatted_output = f"""
# Research Analysis for: {results['topic']}

## Topic Explanation
{explanation}

## Literature Review
{literature}

## Gap Analysis and Research Suggestions
{gaps}
"""

    return formatted_output, explanation, literature, gaps

# Create Gradio UI
with gr.Blocks(title="AI Research Paper Companion") as demo:
    gr.Markdown("""
    # 🔬 AI Research Paper Companion

    This tool uses AI agents to help you understand research topics, find relevant literature, and identify research gaps.

    **Features:**
    - **Topic Explainer**: Provides comprehensive explanations of research topics
    - **Literature Finder**: Searches and summarizes relevant papers
    - **Gap Analyzer**: Identifies research gaps and suggests new directions
    """)

    # Continue with the rest of your Gradio interface code...


In [11]:
def run_research_companion(research_topic, specific_focus=None):
    # Combine topic and focus
    full_topic = research_topic
    if specific_focus:
        full_topic += f" with focus on {specific_focus}"

    # Create and run the crew
    crew = create_research_crew(full_topic)

    try:
        # Execute the crew
        result = crew.kickoff()

        # Parse results
        output = {
            "topic": full_topic,
            "explanation": "",
            "literature_review": "",
            "gap_analysis": "",
            "full_output": str(result)
        }

        # Extract individual agent outputs if available
        if hasattr(result, 'tasks_output'):
            if len(result.tasks_output) >= 3:
                output["explanation"] = str(result.tasks_output[0])
                output["literature_review"] = str(result.tasks_output[1])
                output["gap_analysis"] = str(result.tasks_output[2])

        return output

    except Exception as e:
        return {
            "error": f"An error occurred: {str(e)}",
            "topic": full_topic
        }

# Gradio Interface
def gradio_interface(research_topic, specific_focus, api_key):
    """Gradio interface function"""

    # Update API key if provided
    if api_key and api_key.strip():
        os.environ["OPENAI_API_KEY"] = api_key

    if not research_topic:
        return "Please enter a research topic.", "", "", ""

    # Run the research companion
    results = run_research_companion(research_topic, specific_focus)
    if "error" in results:
        return f"Error: {results['error']}", "", "", ""

    # Format outputs
    explanation = results.get("explanation", results.get("full_output", ""))
    literature = results.get("literature_review", "")
    gaps = results.get("gap_analysis", "")

    return explanation, literature, gaps, results.get("full_output", "")


In [15]:
import os
# Proxy fix
os.environ["NO_PROXY"] = "localhost,127.0.0.1,::1"# Complete updated interface
def gradio_interface(research_topic, specific_focus, api_key):
    if api_key and api_key.strip():
        os.environ["OPENAI_API_KEY"] = api_key
    
    if not research_topic:
        return "Please enter a research topic.", "", "", ""
    
    if not os.getenv("OPENAI_API_KEY"):
        return "Please provide your OpenAI API key.", "", "", ""
    
    results = run_research_companion(research_topic, specific_focus)
    
    if "error" in results:
        error_msg = f"Error: {results['error']}"
        return error_msg, error_msg, error_msg, error_msg
    
    explanation = results.get("explanation", "No explanation generated")
    literature = results.get("literature_review", "No literature review generated") 
    gaps = results.get("gap_analysis", "No gap analysis generated")
    
    full_report = f"""# 🔬 Research Analysis: {results.get('topic', research_topic)}

## 📚 Topic Explanation
{explanation}

## 🔍 Literature Review  
{literature}

## 🎯 Gap Analysis & Research Suggestions
{gaps}
"""
    
    return explanation, literature, gaps, full_report

# Updated Gradio Interface
with gr.Blocks(title="AI Research Paper Companion", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🔬 AI Research Paper Companion
    
    **Powered by AI Agents:** Topic Explainer | Literature Finder | Gap Analyzer
    """)
    
    with gr.Row():
        with gr.Column(scale=2):
            api_key = gr.Textbox(label="🔑 OpenAI API Key", type="password")
            topic = gr.Textbox(label="🎯 Research Topic", placeholder="e.g., Machine Learning, Quantum Computing")
            focus = gr.Textbox(label="🔍 Specific Focus (Optional)", placeholder="e.g., Reinforcement Learning")
            submit = gr.Button("🚀 Start Analysis", variant="primary", size="lg")
    
    with gr.Tab("📊 Agent Results"):
        with gr.Row():
            with gr.Column():
                gr.Markdown("### 📚 Topic Explainer")
                topic_output = gr.Textbox(label="", lines=12, show_label=False)
            
            with gr.Column():
                gr.Markdown("### 🔍 Literature Finder") 
                literature_output = gr.Textbox(label="", lines=12, show_label=False)
            
            with gr.Column():
                gr.Markdown("### 🎯 Gap Analyzer")
                gap_output = gr.Textbox(label="", lines=12, show_label=False)
    
    with gr.Tab("📋 Complete Report"):
        full_output = gr.Markdown("")
    
    submit.click(
        fn=gradio_interface,
        inputs=[topic, focus, api_key],
        outputs=[topic_output, literature_output, gap_output, full_output]
    )

# Sabse easy - port parameter hi remove kar do
demo.launch(
    share=True,
    server_name="0.0.0.0", 
    # server_port=7860,  # Ye line comment kar do
    inbrowser=True,
    debug=True
)




ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7861): only one usage of each socket address (protocol/network address/port) is normally permitted


* Running on local URL:  http://0.0.0.0:7862
* Running on public URL: https://9ea05a16012c9ad148.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 67d15860-cc04-409f-97d0-d4b92e7e5089                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Topic Explainer                                                                                         │
│                                                                                                                 │
│  Task: Provide a comprehensive explanation of the research topic: Machine learning with focus on Reinforcement  │
│  learning                                                                                                       │
│          Include:                                                                                               │
│          1. Core concepts and definitions                                                                       │
│          2. Historical context and evolution                                                                    │
│          3. Current state of the field                                                                          │
│          4. Key challenges and open questions                                                                   │
│          5. Interdisciplinary connections                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Gap Analyzer                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The field of reinforcement learning (RL) presents several research gaps and opportunities. Key gaps include    │
│  the need for data-efficient learning, difficulty in handling tasks with long-term planning or infrequent       │
│  rewards, and a lack of interpretability and transparency. Underexplored areas include the incorporation of     │
│  prior knowledge into RL algorithms and their application in real-world scenarios. Current methodologies face   │
│  limitations such as high computational cost and balancing exploration and exploitation. Interdisciplinary      │
│  collaboration with fields like psychology, finance, healthcare, statistics, and mathematics could offer        │
│  innovative approaches and insights.                                                                            │
│                                                                                                                 │
│  Specific research questions that could be pursued include improving RL algorithms' data efficiency,            │
│  incorporating prior knowledge, designing algorithms for long-term planning tasks, enhancing interpretability   │
│  and transparency, and optimizing the balance between exploration and exploitation. The most promising          │
│  direction for future research would be to enhance data efficiency in RL algorithms. This could involve         │
│  developing novel RL algorithms that learn efficiently from small datasets, potentially using meta-learning or  │
│  transfer learning techniques. Such research would involve comparative studies with existing RL algorithms      │
│  across various tasks and comprehensive analysis and discussion of the results.                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b1afa57e-a822-4379-82ba-6bf39bf305bc                                                                     │
│  Agent: Gap Analyzer                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 67d15860-cc04-409f-97d0-d4b92e7e5089                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The field of reinforcement learning (RL) presents several research gaps and opportunities. Key   │
│  gaps include the need for data-efficient learning, difficulty in handling tasks with long-term planning or     │
│  infrequent rewards, and a lack of interpretability and transparency. Underexplored areas include the           │
│  incorporation of prior knowledge into RL algorithms and their application in real-world scenarios. Current     │
│  methodologies face limitations such as high computational cost and balancing exploration and exploitation.     │
│  Interdisciplinary collaboration with fields like psychology, finance, healthcare, statistics, and mathematics  │
│  could offer innovative approaches and insights.                                                                │
│                                                                                                                 │
│  Specific research questions that could be pursued include improving RL algorithms' data efficiency,            │
│  incorporating prior knowledge, designing algorithms for long-term planning tasks, enhancing interpretability   │
│  and transparency, and optimizing the balance between exploration and exploitation. The most promising          │
│  direction for future research would be to enhance data efficiency in RL algorithms. This could involve         │
│  developing novel RL algorithms that learn efficiently from small datasets, potentially using meta-learning or  │
│  transfer learning techniques. Such research would involve comparative studies with existing RL algorithms      │
│  across various tasks and comprehensive analysis and discussion of the results.                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7861 <> https://f57823b4727d7863b2.gradio.live
Killing tunnel 0.0.0.0:7862 <> https://9ea05a16012c9ad148.gradio.live
